In [ ]:
import jax, flax, numpy
import flax.linen as nn
from notebookinit import *
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import optax
import tiktoken
import lib_kata

2023-12-30 18:22:41.171612: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 18:22:41.171638: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 18:22:41.172542: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 18:22:41.926008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
text_encoded, text_encoder, text_decoder, vocabulary_size = lib_kata.load_dataset_and_tokenize()

vocabulary: 
 ,e,t,o,a,i,h,s,r,n,
,l,d,u,m,y,w,,,c,f,g,b,p,:,k,v,.,',;,?,!,-,j,q,x,z,3,&,$ 
 size: 39 chars
first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [ ]:
context_size=16
embedd_features=5
learning_rate=3e-2

In [ ]:
X_all,y_all = lib_kata.make_training_Xy(text_encoded, context_size=context_size)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.05, shuffle=False, random_state=42)

In [ ]:
lib_kata.preview_Xy(X=X_train, y=y_train, text_decoder=text_decoder, count=5, offset=0)

[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21]  ->  1  |  first citizen:
b _ e
[ 5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1]  ->  19  |  irst citizen:
be _ f
[ 8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19]  ->  3  |  rst citizen:
bef _ o
[ 7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3]  ->  8  |  st citizen:
befo _ r
[ 2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8]  ->  1  |  t citizen:
befor _ e


In [ ]:
vocabulary_size

39

In [ ]:
embedder = nn.Embed(num_embeddings=vocabulary_size, features=5)

In [ ]:
embedder

Embed(
    # attributes
    num_embeddings = 39
    features = 5
    dtype = None
    param_dtype = float32
    embedding_init = init
)

In [ ]:
embedder_example_param = embedder.init(jax.random.PRNGKey(0), X_train[0])

In [ ]:
example_X = X_train[0].reshape([1,context_size])
example_X.shape

(1, 16)

In [ ]:
tabulate_fn = nn.tabulate(embedder, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})

print(tabulate_fn(example_X))

                               Embed Summary                                
┏━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path ┃ module ┃ inputs      ┃ outputs         ┃ params                   ┃
┡━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      │ Embed  │ int32[1,16] │ float32[1,16,5] │ embedding: float32[39,5] │
│      │        │             │                 │                          │
│      │        │             │                 │ 195 (780 B)              │
├──────┼────────┼─────────────┼─────────────────┼──────────────────────────┤
│      │        │             │           Total │ 195 (780 B)              │
└──────┴────────┴─────────────┴─────────────────┴──────────────────────────┘
                                                                            
                       Total Parameters: 195 (780 B)                        

In [ ]:

X_small,y_small = lib_kata.make_training_Xy(text_encoded[0:2000], context_size=context_size)
example_X=X_small[0].reshape([1,context_size])
example_target_y=y_small[0].reshape([-1,1])
example_target_y_logits = nn.activation.one_hot(example_target_y,vocabulary_size)
example_target_y_logits

Array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [ ]:
text_decoder(jnp.argmax(example_target_y_logits).reshape([-1,1]))

'e'

In [ ]:

    
class SimpleNN1(nn.Module):
    # batch size is implied from data
    context_size: int = 8    
    embedd_features: int = 5
    hidden_features_1: int = 16
    hidden_features_2: int = 16
    vocabulary_size: int = 39
    

    def setup(self):
        # print(self.context_size, self.embedd_features, "->", self.context_size * self.embedd_features,)
        pass
    
    @nn.compact
    def __call__(self, x):
        # first, embedd the input
        y0 = nn.Embed(name='embedding',num_embeddings=self.vocabulary_size, features=self.embedd_features)(x)
        # flatten the input vector, as there is no conceptual difference between the token's identity and token's feature here:        
        h = y0.reshape([-1,self.context_size*self.embedd_features])
        
        h = nn.Dense(name='layer_1',features=self.hidden_features_1)(h)
        h = nn.leaky_relu(h)        
        h = nn.Dense(name='layer_2',features=self.hidden_features_2)(h)
        h = nn.leaky_relu(h)        

        
        # final dense layer down to prediction logits. Note that there might be more logits than in the hidden state, that's OK.
        h = nn.Dense(name='layer_final',features=self.vocabulary_size)(h)                
        h = nn.leaky_relu(h)        
        h = nn.activation.softmax(h)        
        y = h
        return y


simpleNN = SimpleNN1(context_size=context_size, embedd_features=embedd_features, hidden_features_1=11, hidden_features_2=7)
simpleNN_params = simpleNN.init(jax.random.PRNGKey(0), example_X)
tabulate_fn = nn.tabulate(simpleNN, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(example_X))
print(f'{simpleNN.apply(simpleNN_params,example_X).shape=}')
predicted_logits = simpleNN.apply(simpleNN_params,example_X)
predicted_example_token = jnp.argmax(simpleNN.apply(simpleNN_params,example_X), axis=1)
predicted_logits_str = ", ".join([f'{float(predicted_logits[1,x]):0.2f}' for x in range(predicted_logits.shape[1])])
print(predicted_logits_str)
print(f'predicted token: ', predicted_example_token, "-> >>>", text_decoder(predicted_example_token),"<<<")
print(f'   target token: ', example_target_y, f"confidence: {float(predicted_logits[0,predicted_example_token][0]):0.3f}")


                                   SimpleNN1 Summary                                    
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path        ┃ module    ┃ inputs        ┃ outputs         ┃ params                   ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             │ SimpleNN1 │ int32[1,16]   │ float32[1,39]   │                          │
├─────────────┼───────────┼───────────────┼─────────────────┼──────────────────────────┤
│ embedding   │ Embed     │ int32[1,16]   │ float32[1,16,5] │ embedding: float32[39,5] │
│             │           │               │                 │                          │
│             │           │               │                 │ 195 (780 B)              │
├─────────────┼───────────┼───────────────┼─────────────────┼──────────────────────────┤
│ layer_1     │ Dense     │ float32[1,80] │ float32[1,11]   │ bias: float32[11]        │
│             │           │               │                 │ kernel: float32[80,11]   │
│             │           │               │                 │                          │
│             │           │               │                 │ 891 (3.6 KB)             │
├─────────────┼───────────┼───────────────┼─────────────────┼──────────────────────────┤
│ layer_2     │ Dense     │ float32[1,11] │ float32[1,7]    │ bias: float32[7]         │
│             │           │               │                 │ kernel: float32[11,7]    │
│             │           │               │                 │                          │
│             │           │               │                 │ 84 (336 B)               │
├─────────────┼───────────┼───────────────┼─────────────────┼──────────────────────────┤
│ layer_final │ Dense     │ float32[1,7]  │ float32[1,39]   │ bias: float32[39]        │
│             │           │               │                 │ kernel: float32[7,39]    │
│             │           │               │                 │                          │
│             │           │               │                 │ 312 (1.2 KB)             │
├─────────────┼───────────┼───────────────┼─────────────────┼──────────────────────────┤
│             │           │               │           Total │ 1,482 (5.9 KB)           │
└─────────────┴───────────┴───────────────┴─────────────────┴──────────────────────────┘
                                                                                        
                            Total Parameters: 1,482 (5.9 KB)                            




simpleNN.apply(simpleNN_params,example_X).shape=(1, 39)
0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03, 0.03
predicted token:  [12] -> >>> d <<<
   target token:  [[1]] confidence: 0.027


In [ ]:
def cross_entropy_loss(logits, labels):
    """Returns cross-entropy loss."""
    assert jnp.all(logits.shape==labels.shape)
    return -jnp.mean(jnp.sum(logits * labels, axis=1))

In [ ]:

jnp.log10(-cross_entropy_loss(example_target_y_logits[0,:,:],predicted_logits))

Array(-1.5991138, dtype=float32)

In [ ]:
cross_entropy_loss(predicted_logits,predicted_logits)

Array(-0.02565163, dtype=float32)

In [ ]:
predicted_logits.shape

(1, 39)

In [ ]:
cross_entropy_loss(predicted_logits,predicted_logits)

Array(-0.02565163, dtype=float32)

In [ ]:
predicted_logits.shape

(1, 39)

In [ ]:
example_target_y_logits.shape

(1, 1, 39)

In [ ]:
float(predicted_logits[0,predicted_example_token][0])

0.026937540620565414

In [ ]:
# I think I am ready to do a training step

In [ ]:
8*5

40

In [ ]:
numpy.prod((4096,8,5))

163840

In [ ]:
numpy.prod((2048,80))

163840

In [ ]:
train_minibatch_size = 32*4096
batches_per_epoch = len(X_train) - train_minibatch_size
print(f'minibatches per epoch: {batches_per_epoch} with minibatch size of {train_minibatch_size}')
starter_X_minibatch = jnp.array(X_train[0:train_minibatch_size])
starter_y_minibatch = jnp.array(X_train[0:train_minibatch_size])

simpleNN_minibatched = SimpleNN1(context_size=context_size, embedd_features=embedd_features, hidden_features_1=11, hidden_features_2=7)
simpleNN_minibatched_params = simpleNN.init(jax.random.PRNGKey(0), starter_X_minibatch)
tabulate_fn = nn.tabulate(simpleNN_minibatched, jax.random.PRNGKey(0), console_kwargs={'width':120, 'force_jupyter':True})
print(tabulate_fn(starter_X_minibatch))

minibatches per epoch: 928537 with minibatch size of 131072


                                        SimpleNN1 Summary                                         
┏━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path        ┃ module    ┃ inputs             ┃ outputs              ┃ params                   ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             │ SimpleNN1 │ int32[131072,16]   │ float32[131072,39]   │                          │
├─────────────┼───────────┼────────────────────┼──────────────────────┼──────────────────────────┤
│ embedding   │ Embed     │ int32[131072,16]   │ float32[131072,16,5] │ embedding: float32[39,5] │
│             │           │                    │                      │                          │
│             │           │                    │                      │ 195 (780 B)              │
├─────────────┼───────────┼────────────────────┼──────────────────────┼──────────────────────────┤
│ layer_1     │ Dense     │ float32[131072,80] │ float32[131072,11]   │ bias: float32[11]        │
│             │           │                    │                      │ kernel: float32[80,11]   │
│             │           │                    │                      │                          │
│             │           │                    │                      │ 891 (3.6 KB)             │
├─────────────┼───────────┼────────────────────┼──────────────────────┼──────────────────────────┤
│ layer_2     │ Dense     │ float32[131072,11] │ float32[131072,7]    │ bias: float32[7]         │
│             │           │                    │                      │ kernel: float32[11,7]    │
│             │           │                    │                      │                          │
│             │           │                    │                      │ 84 (336 B)               │
├─────────────┼───────────┼────────────────────┼──────────────────────┼──────────────────────────┤
│ layer_final │ Dense     │ float32[131072,7]  │ float32[131072,39]   │ bias: float32[39]        │
│             │           │                    │                      │ kernel: float32[7,39]    │
│             │           │                    │                      │                          │
│             │           │                    │                      │ 312 (1.2 KB)             │
├─────────────┼───────────┼────────────────────┼──────────────────────┼──────────────────────────┤
│             │           │                    │                Total │ 1,482 (5.9 KB)           │
└─────────────┴───────────┴────────────────────┴──────────────────────┴──────────────────────────┘
                                                                                                  
                                 Total Parameters: 1,482 (5.9 KB)                                 

In [ ]:
def prep_minibatch(minibatch_idx=0):
    minibatch_ptr = minibatch_idx
    X_minibatch = jnp.array(X_train[minibatch_ptr:minibatch_ptr+train_minibatch_size])
    y_minibatch = jnp.array(y_train[minibatch_ptr:minibatch_ptr+train_minibatch_size])
    y_minibatch_target_logits = nn.activation.one_hot(y_minibatch,vocabulary_size)
    return X_minibatch, y_minibatch_target_logits

In [ ]:
model_starter_params = simpleNN_minibatched_params
model_moving_params = model_starter_params.copy()

In [ ]:
optimizer = optax.adabelief(learning_rate=learning_rate)
optimizer_state = optimizer.init(model_moving_params)

In [ ]:
step_count=0
minibatch_idx = 0

In [ ]:
minibatch_idx

0

In [ ]:
# step
for step_idx in range(20):
    step_count+=1
    minibatch_idx = step_count % batches_per_epoch
    minibatch_X, minibatch_y =prep_minibatch(minibatch_idx=minibatch_idx)
    loss_fn = lambda model_params : -cross_entropy_loss(minibatch_y, simpleNN_minibatched.apply(model_params,minibatch_X))
    loss_value, gradients = jax.value_and_grad(loss_fn)(model_moving_params)
    model_param_updates, optimizer_state = optimizer.update(gradients, optimizer_state, params=model_moving_params)
    step_count+=1
    model_moving_params = optax.apply_updates(model_moving_params, model_param_updates)
    print(f'training loss mag: {numpy.log10(loss_value):0.3f}')

training loss mag: -1.596
training loss mag: -1.634
training loss mag: -1.700
training loss mag: -1.828
training loss mag: -2.085
training loss mag: -2.649
training loss mag: -3.762
training loss mag: -5.082
training loss mag: -6.198
training loss mag: -7.206
training loss mag: -8.203
training loss mag: -9.193
training loss mag: -10.166
training loss mag: -11.115
training loss mag: -12.034
training loss mag: -12.920
training loss mag: -13.770
training loss mag: -14.581
training loss mag: -15.353
training loss mag: -16.085


In [ ]:
minibatch_X[0,:]

Array([ 8,  2,  6,  1,  8, 17,  0,  6,  1,  4,  8,  0, 14,  1,  0,  7],      dtype=int32)

In [ ]:
starter_prompt = jnp.array(X_train[0])

def predict_text(model_params, prompt_encoded, new_characters=50):
    prediction_decoded = text_decoder(prompt_encoded)
    print(f'{prediction_decoded}|||||')
    running_prompt = prompt_encoded.copy()
    # print(running_prompt)
    for char_idx in range(new_characters):
        predicted_logits = simpleNN_minibatched.apply(model_params, running_prompt)
        predicted_token = jnp.argmax(predicted_logits).reshape((1,1))
        predicted_char = text_decoder(predicted_token)
        prediction_decoded = f'{prediction_decoded}{predicted_char}'
        running_prompt = jnp.hstack([running_prompt[1:],predicted_token[:,0]])
        # print(running_prompt)
    print(prediction_decoded)    
predict_text(model_moving_params, starter_prompt)


first citizen:
b|||||
first citizen:
b$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


In [ ]:
raise

RuntimeError: No active exception to reraise

In [ ]:
!git add .
!git commit -m 'savegame. Got loss value and model weight gradients'
